In [2]:
from dataclasses import dataclass
import random
from typing import Callable, List
import pandas as pd
import altair as alt


@dataclass
class TestData:
    """Test data class for testing the performance of the algorithm. The data is a list of integers."""

    name: str
    data: List[int]

    def display(self):
        """Display the data using Altair."""
        df = pd.DataFrame(enumerate(self.data), columns=["index", "value"])
        alt.Chart(df).mark_circle().properties(
            title=self.name, width=1200, height=100
        ).encode(
            alt.X("index:Q"),
            alt.Y("value:Q", scale=alt.Scale(zero=True, domain=[0, 0xFFFFFFFF])),
        ).display()

    def transform(
        self, f: Callable[["TestData"], List[int]], name: str = "transform"
    ) -> "TestData":
        """Transform the data using the function f. The function f should take a TestData object and return a list of integers."""
        return TestData(name=f"{name}({self.name})", data=f(self.data))


TEST_DATA_LEN = 512 // 4 # (512 + 8) * 4 + 12
test_datas = [
    TestData(name="All 0", data=[0x00000000 for _ in range(TEST_DATA_LEN)]),
    TestData(name="All 1", data=[0xFFFFFFFF for _ in range(TEST_DATA_LEN)]),
    TestData(name="increment", data=[(i & 0xFFFFFFFF) for i in range(TEST_DATA_LEN)]),
    TestData(
        name="increment2",
        data=[((i * 0x10000000) & 0xFFFFFFFF) for i in range(TEST_DATA_LEN)],
    ),
    TestData(name="bitwalk", data=[(1 << (i % 32)) for i in range(TEST_DATA_LEN)]),
    TestData(
        name="bitwalk2",
        data=[(~(1 << (i % 32)) & 0xFFFFFFFF) for i in range(TEST_DATA_LEN)],
    ),
    TestData(
        name="random1",
        data=[random.randint(0, 0xFFFFFFFF) for _ in range(TEST_DATA_LEN)],
    ),
    TestData(
        name="random2",
        data=[random.randint(0, 0xFFFFFFFF) for _ in range(TEST_DATA_LEN)],
    ),
]

for test_data in test_datas:
    test_data.display()


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## Bit Scramble

In [3]:
from typing import Generator, List, Optional

class Lfsr32:
    """Linear Feedback Shift Register"""

    def __init__(self, init_value: int = 1, taps: List[int] = [31, 30, 28, 10, 2,1]):
        self.init_value = init_value
        self.value = init_value
        self.taps = taps

    def __tap_bits__(self) -> int:
        return sum([1 << tap for tap in self.taps])

    def reset(self, init_value: Optional[int] = None):
        if init_value is not None:
            init_value = self.init_value
        self.value = self.init_value

    def next(self) -> int:
        tap_bits = self.__tap_bits__()
        self.value = ((self.value >> 1) ^ (-(self.value & 1) & tap_bits)) & 0xFFFFFFFF
        return self.value

    def __str__(self):
        return f"Lfsr32(init={self.init_value}, taps={self.taps})"


def scrample_datas(
    lfsr: Lfsr32,
    data: List[int],
) -> Generator[int, None, None]:
    """Scramble data with LFSR32"""
    lfsr.reset()
    return [d ^ lfsr.next() for d in data]


# test
lfsr = Lfsr32()
scramble_test_datas = [
    test_data.transform(lambda x: scrample_datas(lfsr, x), name=f"xor {lfsr}")
    for test_data in test_datas
]
for test_data in scramble_test_datas:
    test_data.display()

# decode & verify
descramble_test_datas = [
    test_data.transform(lambda x: scrample_datas(lfsr, x), name=f"xor {lfsr}")
    for test_data in scramble_test_datas
]
for test_data in descramble_test_datas:
    test_data.display()
for src, dst in zip(test_datas, descramble_test_datas):
    assert src.data == dst.data




alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## ECC

- https://www.akita-pu.ac.jp/system/elect/ins/kusakari/japanese/teaching/InfoTheo/2009/note/10.pdf
- https://prml.main.ist.hokudai.ac.jp/wp-content/uploads/2020/07/infth2020-13.pdf

In [134]:
def hamming_generator_matrix(r):
    n = 2**r - 1
    k = n - r

    # 生成行列の初期化
    G = [[0 for x in range(n)] for y in range(k)] 

    # パリティビットとデータビットの位置を決定
    for i in range(n):
        if i & (i + 1) != 0:  # 2の冪乗でない場合
            for j in range(k):
                if i >= r and ((j >> (i - r)) & 1):  # jのiビット目が1の場合
                    G[j][i] = 1

    return G

r = 5
G = hamming_generator_matrix(r)

def generate_syndrome_table(G):
    # G is the generator matrix
    n = len(G[0])  # length of codeword
    k = len(G)  # length of message
    syndrome_table = {}

    for i in range(1, 2**n):
        syndrome = [0]*n
        for j in range(n):
            if (i & (1 << j)) != 0:  # if jth bit of i is set
                syndrome = [syndrome[x] ^ G[x][j] for x in range(k)]
        syndrome_table["".join(map(str, syndrome))] = i

    return syndrome_table

syndrome_table = generate_syndrome_table(G)
syndrome_table

KeyboardInterrupt: 

In [141]:
import copy
from enum import IntEnum, auto
import functools
from dataclasses import dataclass, field
import math
import os
import pprint
from typing import Dict


@dataclass
class EccUnit:
    """ParityとParity生成元になったデータ"""

    data_src: List[int]
    parity: int
    checked_bits: int
    padding_bits: int

    @classmethod
    def init(cls):
        return cls(data_src=[], parity=0, checked_bits=0, padding_bits=0)


class EccDecodeStatus(IntEnum):
    """decode結果"""

    Ok = auto()
    Correctable = auto()
    Uncorrectable = auto()


@dataclass
class EccDecodeResult:
    """decode結果とその詳細"""

    result: EccDecodeStatus
    data: List[int]
    syndrome: int
    calc_ecc_unit: EccUnit

    error_bitpos: Optional[int] = None


@dataclass
class EccProcessor:
    """Error correction code, 32bit CPU実装最適化向けに4byte単位で処理"""

    data_bit_n: int
    parity_bit_n: int
    include_even_parity: bool = True
    process_unit_bytes: int = 4

    # private
    generator_list: List[int] = field(default=None)
    syndrome_list: List[int] = field(default=None)
    syndrome_bitpos_dict: Dict[int, int] = field(default=None)

    def __post_init__(self):
        self.generator_list = list(self.__create_generator())
        self.syndrome_list = list(self.__create_syndrome_list())
        # 訂正時、Syndromeの値から位置を求めるのでindex->syndromeからsyndrome->indexのdictに変換しておく
        self.syndrome_bitpos_dict = dict(
            [(syndrome, bitpos) for bitpos, syndrome in enumerate(self.syndrome_list)]
        )
        assert len(self.syndrome_bitpos_dict) > self.data_bit_n, "syndrome_bitpos_dict is too short"

    @property
    def __process_unit_bits(self) -> int:
        """処理単位のbit数"""
        return self.process_unit_bytes * 8

    @property
    def __data_unit_mask(self) -> int:
        return int("0x" + ("ff" * self.process_unit_bytes), 16)

    @property
    def __total_bits(self) -> int:
        """data+parityの総bit数"""
        return self.data_bit_n + self.parity_bit_n

    @property
    def __data_unit_n(self) -> int:
        """処理単位のデータ数"""
        return math.ceil(self.__total_bits / self.__process_unit_bits)

    def pretty_str(self) -> str:
        """検査bit確認用"""
        return f"{'SECDEC' if self.include_even_parity else 'SEC'}({self.__total_bits}, {self.data_bit_n})\n- process_unit: {self.process_unit_bytes} byte\n- checkbit_mask\n{''.join(['  - ' + format(x, '032b') + os.linesep for x in ecc.__create_generator()])}\n"

    def __create_generator(self) -> Generator[int, None, None]:
        """
        `process_unit_bytes` ごとのデータに対し、各Parity bitが検査を行うbitに1を立てたMaskを返す
        """
        r = self.parity_bit_n - 1 if self.include_even_parity else self.parity_bit_n
        print(r)
        # 総ビット数
        n = 2**r - 1
        # データビット数
        k = n - r

        # 生成行列の初期化
        G = [[0 for x in range(n)] for y in range(k)] 

        # パリティビットとデータビットの位置を決定
        for i in range(n):
            if i & (i + 1) != 0:  # 2の冪乗でない場合
                for j in range(k):
                    if i >= r and ((j >> (i - r)) & 1):  # jのiビット目が1の場合
                        G[j][i] = 1

        # 2進数文字列に戻して数値化
        for row in G:
            bin_str = "0b" + "".join([str(x) for x in reversed(row)])
            yield int(bin_str, 2)

        # 拡張ハミング符号では符号語全体のParitybitを追加。全データxor = even parity
        if self.include_even_parity:
            yield self.__data_unit_mask

    def __xor_checkbit(self, data: int) -> int:
        """指定されたデータをxorしてparity bitを返す。実処理はこのparity bitを更にxorして累積する"""
        parity = 0
        for i, checkbit in enumerate(self.generator_list):
            # xor mask bit & reduce(xor) to 1bit
            # C/Rust実装するなら__builtin_popcountあたり使えばよいかも. (__builtin_popcount(data & checkbit) & 0x1)
            parity ^= (bin(data & checkbit).count("1") & 0x1) << i
        return parity

    def __create_syndrome_list(self) -> Generator[int, None, None]:
        """検査行列の内容を参照し、parity bitの組み合わせに対するdatabit配置を求める。bit0から昇順で生成"""
        assert self.generator_list is not None, "checkbit_list is None"
        print(f"len(self.checkbit_list)= {len(self.generator_list)}")

        # 取りうるデータ全部
        for data_error_bitpos in range(0, self.data_bit_n):
            syndrome = 0
            # data_error_bitpos だけに1を含むerror vector
            error_unit_pos = data_error_bitpos // self.__process_unit_bits
            error_bit_offset = data_error_bitpos % self.__process_unit_bits
            data_src = list(
                [
                    (1 << error_bit_offset) if i == error_unit_pos else 0
                    for i in range(self.__data_unit_n)
                ]
            )
            for data in data_src:
                syndrome ^= self.__xor_checkbit(data)
            print(f"data_error_bitpos:{data_error_bitpos:32d} -> syndrome:{syndrome:032b}")
            assert syndrome != 0, f"syndrome is 0 at {data_error_bitpos:032b}"
            yield syndrome

    def __encode_unit(self, ecc_unit: EccUnit, data: int, data_bits: int):
        """EccUnitへの値追加用の内部関数"""
        ecc_unit.parity ^= self.__xor_checkbit(data)
        ecc_unit.checked_bits += data_bits
        ecc_unit.data_src.append(data)

    def encode(
        self, datas: Generator[int, None, None]
    ) -> Generator[EccUnit, None, None]:
        """
        Encodeを行う
        - datas: `process_unit_bytes` byteごとのデータのリスト
        - return: Parity bitのリスト
        """
        ecc_unit = EccUnit.init()

        for data in datas:
            if ecc_unit.checked_bits + self.__process_unit_bits < self.data_bit_n:
                # 4byteキャプチャしてもまだ末尾には到達しない
                self.__encode_unit(
                    ecc_unit=ecc_unit, data=data, data_bits=self.__process_unit_bits
                )
            else:
                # 現在のParityに含めるデータと、残りのデータの2分割して処理. LSBから処理する
                unchecked_bits = self.data_bit_n - ecc_unit.checked_bits
                encode_data_unit0 = (
                    data
                    & (~(self.__data_unit_mask << unchecked_bits))
                    & self.__data_unit_mask
                )
                encode_data_unit1 = (data >> unchecked_bits) & self.__data_unit_mask
                assert (
                    encode_data_unit0 | (encode_data_unit1 << unchecked_bits) == data
                ), f"encode_data_unit0({encode_data_unit0:32b}) | (encode_data_unit1({encode_data_unit1:032b}) << {unchecked_bits}) != data({data:032b})"

                # unit0
                self.__encode_unit(
                    ecc_unit=ecc_unit, data=encode_data_unit0, data_bits=unchecked_bits
                )
                yield ecc_unit
                # reset
                ecc_unit = EccUnit.init()
                # unit1
                # 残りのデータがprocess_unit_bytes未満の場合のみ処理必要
                if unchecked_bits < self.__process_unit_bits:
                    self.__encode_unit(
                        ecc_unit=ecc_unit,
                        data=encode_data_unit1,
                        data_bits=self.__process_unit_bits - unchecked_bits,
                    )
        # 入力データが終わったが、parity bitが残っている場合
        if ecc_unit.checked_bits > 0:
            ecc_unit.padding_bits = self.data_bit_n - ecc_unit.checked_bits
            yield ecc_unit

    def __calc_correct_pos(self, syndrome: int) -> Optional[int]:
        """
        syndromeから訂正すべきbit位置を返す
        """

        # 訂正不要
        if syndrome == 0:
            return None

        # dictで事前に変換済
        correct_bitpos = self.syndrome_bitpos_dict.get(syndrome)
        return correct_bitpos

    def decode(
        self,
        datas: Generator[int, None, None],
        ref_parity_datas: Generator[int, None, None],
    ) -> Generator[EccDecodeResult, None, None]:
        """
        Decodeを行う。Data bits, parity bitsは別々に格納して、これを下にエラーが有るかどうかを優先チェックする
        - datas: `process_unit_bytes` byteごとのデータのリスト
        - ref_parity_datas: Parity bitのリスト
        - return: エラー有無のリスト
        """
        # dataからParityを計算
        calced_parity_datas = self.encode(datas)

        for calc_p, ref_p in zip(calced_parity_datas, ref_parity_datas):
            src_data = calc_p.data_src
            syndrome = calc_p.parity ^ ref_p.parity
            # 訂正不要
            if syndrome == 0:
                yield EccDecodeResult(
                    result=EccDecodeStatus.Ok,
                    data=src_data,
                    calc_ecc_unit=ref_p,
                    syndrome=syndrome,
                )
                continue
            # 要訂正
            correct_bitpos = self.__calc_correct_pos(syndrome)
            assert correct_bitpos is not None, "correct_bitpos is None"
            correct_dataunit_pos = correct_bitpos // self.__process_unit_bits
            correct_bitoffset = correct_bitpos % self.__process_unit_bits

            correct_data = copy.deepcopy(src_data)
            correct_data[correct_dataunit_pos] ^= 1 << correct_bitoffset

            yield EccDecodeResult(
                result=EccDecodeStatus.Correctable,
                data=correct_data,
                calc_ecc_unit=ref_p,
                syndrome=syndrome,
                error_bitpos=correct_bitpos,
            )

    def unpack_decode_result(
        self, decode_results: Generator[EccDecodeResult, None, None]
    ) -> List[int]:
        """decode結果を展開してデータのリストを返す"""
        return list([d for decode_result in decode_results for d in decode_result.data])


ecc = EccProcessor(
    data_bit_n=247, parity_bit_n=9, include_even_parity=True, process_unit_bytes=4
)
print(ecc.pretty_str())

origin_data = list([i for i in range(0, 32)])
origin_parity = list(ecc.encode(origin_data))

# 247bitごと1回だけ入れないと無理
# corrupt_1bit_data = list([d ^ (1 << i) if i % 16 == 0 else d for i, d in enumerate(origin_data)])
corrupt_data = copy.deepcopy(origin_data)
corrupt_data[0] ^= 0x00000010


correct_results = list(ecc.decode(datas=corrupt_data, ref_parity_datas=origin_parity))
correct_data = list(ecc.unpack_decode_result(correct_results))

print(f"origin:  {list(origin_data)}")
print(f"corrupt: {list(corrupt_data)}")
print(f"correct: {list(correct_data)}")

for origin_d, corrupt_d, correct_d, correct_results in zip(
    origin_data, corrupt_data, correct_data, correct_results
):
    pprint.pp(
        f"origin={origin_d:032b}, corrupt={corrupt_d:032b}, correct={correct_d:032b}, info={correct_results}"
    )
    assert origin_d == correct_d


8
len(self.checkbit_list)= 248
data_error_bitpos:                               0 -> syndrome:10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
data_error_bitpos:                               1 -> syndrome:10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
data_error_bitpos:                               2 -> syndrome:10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
data_error_bitpos:               

AssertionError: syndrome_bitpos_dict is too short